# Introduction

# Motivation

In [1]:
# Import Libraries

import azureml.core
from azureml.core import Workspace

from azureml.core import Dataset
from azureml.data.datapath import DataPath

import os

## Connect to Workspace

In [2]:

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Ready to use Azure ML 1.48.0 to work with testerinos


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FB6KG4B4L to authenticate.


In [3]:


default_ds = ws.get_default_datastore()

if 'titanic dataset' not in ws.datasets:
    Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'titanic-data/')
                              )

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'titanic-data/titanic.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='titanic dataset',
                                description='titanic data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

2023-02-24 16:17:23.022118 | ActivityCompleted: Activity=get_by_name, HowEnded=Failure, Duration=511.71 [ms], Info = {'activity_id': '67a03579-24c2-4a70-8218-d922fac63bdf', 'activity_name': 'get_by_name', 'activity_type': 'PublicApi', 'app_name': 'dataset', 'source': 'azureml.dataset', 'version': '1.48.0', 'dataprepVersion': '4.8.4', 'sparkVersion': '', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': '', 'completionStatus': 'Success', 'durationMs': 0.09}, Exception=UserErrorException; UserErrorException:
	Message: Cannot find dataset registered with name "titanic dataset" (version: None) in the workspace.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot find dataset registered with name \"titanic dataset\" (version: None) in the workspace."
    }
}
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info

Validating arguments.
Arguments validated.
Uploading file to titanic-data/
Uploading an estimated of 3 files
Uploading data/.amlignore
Uploaded data/.amlignore, 1 files out of an estimated total of 3
Uploading data/titanic.csv
Uploaded data/titanic.csv, 2 files out of an estimated total of 3
Uploading data/.amlignore.amltmp
Uploaded data/.amlignore.amltmp, 3 files out of an estimated total of 3
Uploaded 3 files
Creating new dataset


In [6]:

# Create a folder for the pipeline step files
experiment_folder = 'titanic_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

titanic_pipeline


In [ ]:
%%writefile $experiment_folder/prep_titanic.py
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
save_folder = args.prepped_data

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['raw_data'].to_pandas_dataframe()

# Log raw row count
row_count = (len(diabetes))
run.log('raw_rows', row_count)

# remove nulls
diabetes = diabetes.dropna()

# Normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree']
diabetes[num_cols] = scaler.fit_transform(diabetes[num_cols])

# Log processed rows
row_count = (len(diabetes))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
diabetes.to_csv(save_path, index=False, header=True)

# End the run
run.complete()